# Computer Homework 2: Electric Fields and Gauss's Law

In this assignment we will review Gauss' law (equivalent to Coulomb's law) and understand its limitations.  We will then use the superposition principle and Coulomb's law to calculate the field of a spherical charge distribution and a cube of charge.  We will compare these solutions to each other and to the analytic solution for a spherical charge distribution from Gauss' law.

## Organization of this notebook:

This notebook is divided into three sections.  We first review Gauss' law, next we review how to do integration with Python (you will need this to apply the superposition principle to compute the field of the sphere and cube).  The final section is the two assignments.  The first assignment, homework 3, is to calculate the electric field.  In the next assignment, we will compute the electric potential.

## Gauss' Law

Gauss' law is mathematically equivalent to Coulomb's law.  It expresses a relation between the electrostatic flux ($\Phi_E$) through a closed surface and the charge enclosed ($Q$) through the following relation:  $$\Phi_E \equiv \oint_{surface} \vec E \cdot d\vec A  = \frac {Q_{encl}} {\epsilon_0}. $$  It is often convenient to remember that the charge enclosed can be expressed as an integral over the volume enclosed by the bounding surface of Gauss' law: $Q_{encl} = \iiint_{volume} \rho dV$.  

You have complete choice over which surface to integrate over.  This freedom can be used to make arguments about the distribution of charges in conductors or to calculate the field in cases with a high degree of symmetry.   To calculate $\vec E$ using this law you must find a surface on which the electric field magnitude is constant and the field is always normal to the surface.   This means you need either continuous translation symmetry (e.g., a plane or the axial direction of a cylinder), or/and a continuous rotation symmetry (eg a specially symmetric charge distribution or the azimuthal direction of a cylindrically symmetric charge distribution).  In cases which have these symmetries this is the best way to calculate the field.  

However, there are many charge distributions that don't have this symmetry.  In that case you must resort to other techniques including superposition. For continuous charge distributions, superposition takes the form of an integral.

### Superposition
In the previous exercise we presented superposition as the rule that the field from two souerces is the vector sum of both.  It is easy to extend this to a continuous distribution of charges:  $$\vec E(\vec r)  = \frac {1}{4\pi\epsilon_0}  \iiint_{\mathrm{charge~distribution}} \frac {\hat {\mathscr r}}{{\mathscr r}^2}  {\rho dV}.$$  

It is important to note that $\vec {\mathscr r} = \vec r - \vec r_{charge} $.  In other words, $\vec {\mathscr r}$ depends on the variables you are integrating over.


### Numerical Integration
For the purposes of numerical computation recall that the integral is defined as the limit of a Riemann sum:  $\iiint f(r) dV = \lim_{\Delta V \to 0}  \sum_{\mathrm{volume~elements}} f(r) \Delta V$.  In other words an integral is calculated by dividing the integration volume into lots of small chunks, evaluating the function at the center of these chunks, multiplying them by the volume of the chunk and then summing over all chunks.  With a computer we can't make the chunks infinitely small, but if we make them small enough we can calculate the integral to sufficient precision.   (Lots of care is needed in choosing what precision is needed and verifying that you have met that requirement.)
### NumPy
We will be using routines from the NumPy library.  The users guide for NumPy is at http://docs.scipy.org/doc/numpy/user/
### The first part of the code is to import the usual libraries

In [ ]:
# numpy (NUMerical PYthon) is a library that inclues most of the numerical functionality you will need
import numpy as np                  
# matplotlib (MAThematical PLOTting LIBrary) is the library we use to plot
import matplotlib.pyplot as plt

## Example 1
## Calculate the total charge contained in a 1m$^{3}$ cube with a charge density of 1C/m$^{3}$

The answer is obvious, but this is a great way to show you how to perform a numerical integration.  We want to validate the numeric algorithm, and the easiest way to do this is start with a problem that is easy to code, and compare the result to a known answer.

The first step is to create arrays that give the $x$, $y$, and $z$ position of the elements of our charge distribution.  We have freedom to choose a step size for the grid.  In making this choise we must be careful that the steps are small enough that the function doesn't change much but not so small that the contribution of the element is not lost in the precision of the addition.

Documentation on np.meshgrid can be found at http://docs.scipy.org/doc/numpy/reference/generated/numpy.meshgrid.html?highlight=meshgrid#numpy.meshgrid
and documentation on np.arange can be found at
http://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html?highlight=arange#numpy.arange

In [ ]:
linear_step = 0.05  ## this is the step size expressed as a linear dimension.
negative_cube_edge = -0.5  ## low boundary of the cube in x,y, and z
positive_cube_edge = 0.5   ## upper boundary ... 

x_charge, y_charge, z_charge = np.meshgrid(
                                np.arange(negative_cube_edge, positive_cube_edge, linear_step),
                                np.arange(negative_cube_edge, positive_cube_edge, linear_step),
                                np.arange(negative_cube_edge, positive_cube_edge, linear_step)
                                )

print("shape of the x_charge array: ", np.shape(x_charge))
print("number of elements in the x_charge array: ", np.size(x_charge))

This creates three arrays that each have 8000 elements representing 20 steps in $x, y$, and $z$. These are three dimensional arrays.

The next step is to compute the charge in each cell.  The volume element associated with each grid point is just the "linear_step" variable cubed.  So the charge in each cell is just the charge density times the volume element.

In [ ]:
rho = 1.
d_vol = linear_step**3.

## now make an array that holds the charge of each grid point
dq = np.ones(np.shape(x_charge)) * rho * d_vol  ## this makes an array of the same 
                                          # size as x (and y and z)
                                          # with all elements set to rho * d_vol
print("Shape of the dq array: ", np.shape(dq))

The ```np.ones()``` function creates an array with every element set to 1.  The function ```np.shape()``` returns the array shape of its argument.   Thus 
```
np.ones(np.shape(x_charge))
```
creates an array of the same shape of ```x_charge```, but with all elements set to 1.

*Short Exercise:* In the cell below output an np.ones() object with shape (2,2) to see an example

In [ ]:
# Your example np.ones() here

The next and final step is to do the integration.  This is wonderfully straightforward.

In [ ]:
## this is the triple integral
total_charge = np.sum(dq)      ## could it be any easier to do a triple integral?
    
print("This is the total charge: ",total_charge, " consistent with our expectation it would be 1.")

The notation make the triple integral look trivial, but remember that the hard work was all in setting up the arrays of the position variables and dq.  And because computers love doing this, they feel a special kinship with electric fields.

## Example 2
## Calculate the total charge continued within a sphere with 1 m diameter and a charge density of 1C/m$^{3}$

All we have to do is take the cubic grid from the first example, but only sum over the charges contained within a sphere.  We can do this with the "where" command. 

In [ ]:
## calculate the radius from the origin
r = np.sqrt(x_charge**2 + y_charge**2 + z_charge**2)

## test for the points that are within 0.5 m of the origin
inside_sphere = np.where(r <= 0.5) 

## do the triple integral
total_charge = np.sum(dq[inside_sphere])   ## the brackets subscript dq with only the elements inside
                                           ## has the triple integral over a sphere ever been easier?
print("This is the total charge: ",total_charge)
print("the analytic integral gives: ", 4/3. * np.pi * 0.5**3.)
print("so we have this to ",round((total_charge - 4/3. * np.pi * 0.5**3.)/total_charge*100,2),"% accuracy")

## Miscellaneous useful code tricks 
### (1) Recall from assignment 1, if you add (or subtract) a number to an array python adds that number to each element in the array

As an example, if we have a reference position (x,y,z) and we want the separation vector between this point and the grid positions that describe our charge distribution we can do the following:

In [ ]:
## reference position, that is the point r where we are going to find the electric field
x = 10.
y = 1.
z = 0.

## these are the componets of the separation vetor r_sep = r - r_charge
x_sep = x - x_charge
y_sep = y - y_charge
z_sep = z - z_charge

## print out the size
print("Don't forget these are arrays.  For example x_sep has ", np.size(x_sep), "elements")

### (2) The for loop 
You can perform a loop (i.e. iterate) over any python object that is an iterable, performing an action on each iteration.  A numpy array is one example of an iterable python object.  Let us take a numpy array ranging from -2 to 2, with values spaced by 0.5. You can do the following to loop over all the elements, with the action of printing each element:

In [ ]:
x_field = np.arange(-2,2,.5)

for x in x_field:
    print(x)

## you can play tricks to make this work on 2d arrays too.

## Homework 2 [40 points total] begins here:

### The field of continuous charge distributions
We can use the superposition principle or Gauss's law to calculate the field of a continuous charge distribution.  In this exercise we will use both for the sphere and then use superposition to calculate the field of a cube of charge.

First use Guass' law to calculate the field of a solid insulating sphere with a radius of  0.5 m with charge density 1 C /  m$^{3}$.  There is no charge outside this sphere.  Your answer should include the field inside and outside the sphere.  Plot the field as a function of distance from the center of the sphere.  You will use this to validate your calculations.

#### (Part 1, 10 points)   Use the superposition principle and gauss' law to calculate the field from the sphere described above, using numerical integration and check that the results are the same as your calculation.
Since these calculations are time consuming find the components of the field along the x axis in the range from -3 to 3 m.  Full credit for a correct calculation and a plot (or plots) that show the x,y, and z components of your field agree with the analytic solution.


In [ ]:
## your code should go here.

#### (Part 2, 10 points)   Adapt the code from part 1 to calculate the field of a cube with side length 1 m that is centered at the origin and has the same charge density, that is density 1 C /  m$^{3}$.  
Calculate the field of the cube at the same positions as the sphere and compare the two results.  Full credit for a correct calculation and a plot (or plots) that show the x,y, and z components of the field.



#### (Part 3, 10 points) make a vector plot in the x-y plane of the field of the sphere and the field of the cube.  

Full credit for nice looking 2d vector plots of the field for both charge configurations.

In [ ]:
## your code should go here.

### a few hints to make this easy

# One way to do this is to adapt the code for the field of a 2d square sheet that
# we discussed in class this week.

## Another way is to loop over the field points in your grid.
## This makes two two dimensional arrays
x, y = np.meshgrid(np.arange(-2, 2, 0.1),
                   np.arange(-2, 2, 0.1))
## you can flatten these (convert them to 1d arrays) using ravel
x = np.ravel(x)
y = np.ravel(y)
## then you can make an array to index these position variables and proceed from here.
index = np.arange(np.size(x))


#### (Part 4, 10 points)   Discussion of the results.
Is the electric field from one of the charge distributions always larger than the other, or does it depend on r? Explain physically why this result makes sense. What do you expect would happen if you looked at Ex for still larger r? Will the two curves approach each other, or do you expect the fractional difference to remain non-zero? Is there a size of cube that would make the asymptotic fields the same for the cube and sphere?

Can you use Gauss' law to calculate the field of a cube?  Why or why not?


Double click to write your discussion here.